## Imports

In [84]:
import numpy as np
from sklearn.linear_model import LinearRegression

from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

## Higher Order functions

In [85]:
# series of functions performing exponentiation to given power
def square(x):
    return x**2

def cube(x):
    return x**3

# make exponent and explicit argument
def generic_exp(x,exponent): # requires specify the exponent everytime you call the function
    return x**exponent



We need higher order function(function builder) to build functions (square,cube ...)

In [86]:
def skeleton_exponentiation(x):
    return x**exponent

In [87]:
# skeleton_exponentiation(2)

<span style="color:red">---------------------------------------------------------------------------</span>  
<span style="color:red">NameError</span>                                 Traceback (most recent call last)  
Cell In[9], line 1  
<span style="color:green">----> 1 skeleton_exponentiation(2)</span>  
  
Cell In[8], line 2  
      1 def skeleton_exponentiation(x):  
<span style="color:green">----> 2     return x**exponent</span>  
  
<span style="color:red">NameError</span>: name 'exponent' is not defined  

In [88]:
def exponentiation_builder(exponent):
    def skeleton_exponentiation(x):
        return x**exponent

    return skeleton_exponentiation

In [89]:
returned_function=exponentiation_builder(2)
returned_function

<function __main__.exponentiation_builder.<locals>.skeleton_exponentiation(x)>

In [90]:
exponentiation_builder(2)(4)

16

In [91]:
returned_function(5)

25

In [92]:
cube=exponentiation_builder(3)
cube(5)

125

In [116]:
# Helper functiom 1
def make_train_step(model,loss_fn,optimizer): 
    # Build a function that perform a step in the train loop
    def perform_train_step_fn(x,y):

        # set model to train state
        model.train()

        # forward pass
        y_hat=model(x)

        # compute loss
        loss=loss_fn(y_hat,y)

        # compute gradients
        loss.backward()

        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    # return the function that will be called inside the train loop
    return perform_train_step_fn 

Run: Data preperation 

In [94]:
%run -i data_generation/simple_linear_regression.py

<Figure size 640x480 with 0 Axes>

In [95]:
%run -i data_preparation/v0.py

In [117]:
%%writefile model_configuration/v1.py

device='cuda' if torch.cuda.is_available() else 'cpu'
lr=0.01

torch.manual_seed(42)
# create a model and send it to device
model=nn.Sequential(nn.Linear(1,1)).to(device)

# define SGD optmizer
optimizer=optim.SGD(model.parameters(),lr=lr)

# define loss function
loss_fn=nn.MSELoss(reduction='mean')

# create train step function
train_step_fn=make_train_step(model,loss_fn,optimizer)


Overwriting model_configuration/v1.py


In [118]:
%run -i model_configuration/v1.py

In [98]:
train_step_fn

<function __main__.make_train_step.<locals>.perform_train_step_fn(x, y)>

In [119]:
%%writefile model_training/v1.py

n_epochs=10000
losses=[]

for epoch in tqdm(range(n_epochs)):

    # perform train step and return corresponding loss
    loss=train_step_fn(X_train_tensor,y_train_tensor) # perform one training step
    losses.append(loss) # keep track of loss

Overwriting model_training/v1.py


In [120]:
%run -i model_training/v1.py

  0%|          | 0/10000 [00:00<?, ?it/s]

In [121]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9689]])), ('0.bias', tensor([1.0236]))])

## Dataset

In [133]:
class CustomDataset(Dataset):
    # takes whatever arguments that need to create a list of tuples
    def __init__(self,x_tensor,y_tensor):
        self.x=x_tensor
        self.y=y_tensor

    # return a tuple corresponding to the index, load data on demand
    def __getitem__(self,index):
        return (self.x[index],self.y[index])

    # return the size of the dataset
    def __len__(self):
        return len(self.x)

X_train_tensor=torch.as_tensor(X_train).float() # we don't want to store whole training data into GPU tensors, it will eat up vram
y_train_tensor=torch.as_tensor(y_train).float()

train_data=CustomDataset(X_train_tensor,y_train_tensor)
print(train_data[[1,2,4]])

(tensor([[0.0636],
        [0.8631],
        [0.7320]]), tensor([[1.1928],
        [2.9128],
        [2.4732]]))


## Tensor Dataset

In [135]:
train_data=TensorDataset(X_train_tensor,y_train_tensor) # if dataset is couple of tensors, use TensorDataset
print(train_data[[1,2,4]])

(tensor([[0.0636],
        [0.8631],
        [0.7320]]), tensor([[1.1928],
        [2.9128],
        [2.4732]]))


## Dataloader

In [172]:
# Dataloader is an iterator that will load data on demand
train_loader=DataLoader(dataset=train_data,batch_size=16,shuffle=True)
train_loader

In [173]:
next(iter(train_loader)) # this will load the first batch of data

[tensor([[0.4722],
         [0.8631],
         [0.0636],
         [0.8662],
         [0.4952],
         [0.0740],
         [0.5613],
         [0.4561],
         [0.4938],
         [0.9699],
         [0.5248],
         [0.7852],
         [0.0977],
         [0.7081],
         [0.1196],
         [0.0452]]),
 tensor([[1.9857],
         [2.9128],
         [1.1928],
         [2.6805],
         [1.8735],
         [1.1713],
         [2.0472],
         [1.7706],
         [1.9060],
         [2.9727],
         [2.0167],
         [2.5283],
         [1.4417],
         [2.3660],
         [1.3214],
         [0.9985]])]

In [174]:
list(train_loader) # this will load all the data

[[tensor([[0.9219],
          [0.3252],
          [0.0254],
          [0.8324],
          [0.5427],
          [0.4561],
          [0.7132],
          [0.3745],
          [0.6376],
          [0.7296],
          [0.6842],
          [0.4402],
          [0.3585],
          [0.4952],
          [0.7751],
          [0.9395]]),
  tensor([[2.8506],
          [1.7291],
          [1.0785],
          [2.6119],
          [2.2161],
          [1.7706],
          [2.6162],
          [1.7578],
          [2.1930],
          [2.5751],
          [2.3492],
          [1.9105],
          [1.7462],
          [1.8735],
          [2.4936],
          [2.8890]])],
 [tensor([[0.7069],
          [0.0581],
          [0.0651],
          [0.1560],
          [0.0206],
          [0.1987],
          [0.9869],
          [0.3309],
          [0.8948],
          [0.5613],
          [0.7081],
          [0.0977],
          [0.8662],
          [0.9696],
          [0.6075],
          [0.4722]]),
  tensor([[2.4388],
          [1.

We need to add Dataloader and Dataset elements into Data prep file

In [175]:
%%writefile data_preparation/v1.py

X_train_tensor=torch.as_tensor(X_train).float()
y_train_tensor=torch.as_tensor(y_train).float()

# Builds Dataset
train_data=TensorDataset(X_train_tensor,y_train_tensor)

# Build Dataloader
train_loader=DataLoader(dataset=train_data,batch_size=16,shuffle=True)


Writing data_preparation/v1.py


In [176]:
%run -i data_preparation/v1.py

Now we need to introduce mini batch gradient descent to model training part

In [177]:
%run -i model_configuration/v1.py

In [178]:
%%writefile model_training/v2.py

# Define number of epochs
n_epochs=10000

losses=[]

for epoch in tqdm(range(n_epochs)):
    # inner loop
    mini_batch_losses=[]
    for x_batch,y_batch in train_loader:
        # the dataset lives on CPU, we need to send mini-batches to the device where our model lives
        x_batch=x_batch.to(device)
        y_batch=y_batch.to(device)
        
        # perform training step and return corresponding loss
        mini_batch_loss=train_step_fn(x_batch,y_batch)
        mini_batch_losses.append(mini_batch_loss)

    # compute average loss over all mini batches
    loss=np.mean(mini_batch_losses)
    losses.append(loss)


Writing model_training/v2.py


In [179]:
%run -i model_training/v2.py

  0%|          | 0/10000 [00:00<?, ?it/s]

In [180]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9690]])), ('0.bias', tensor([1.0236]))])